In [76]:
%load_ext autoreload
%autoreload 2

from shapely.geometry import LineString, Point, MultiPoint, Polygon

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import geopandas as gpd
import numpy as np
from datetime import datetime
from shapely.geometry import LineString, Point, MultiPoint, Polygon
from IPython.display import display, clear_output


df = pd.read_csv('data/train.csv')

starts = gpd.GeoDataFrame(df[['id']], geometry=gpd.points_from_xy(df.pickup_longitude, df.pickup_latitude), crs=4326)
stops = gpd.GeoDataFrame(df[['id']], geometry=gpd.points_from_xy(df.dropoff_longitude, df.dropoff_latitude), crs=4326)

path_to_data = gpd.datasets.get_path("nybb")
gdf = gpd.read_file(path_to_data)
manh = gdf.loc[gdf.BoroName=='Manhattan'].to_crs(4326)

roads = gpd.read_file('data/nyc_roads/geo_export_05dcab6d-50ed-427f-aae3-61ecc2069210.shp', mask=manh)

sind = gpd.tools.sjoin(starts, manh, op='within').index.tolist()
eind = gpd.tools.sjoin(stops, manh, op='within').index.tolist()
df = df.loc[list(set(sind).intersection(set(eind)))]
starts = gpd.GeoDataFrame(df[['id']], geometry=gpd.points_from_xy(df.pickup_longitude, df.pickup_latitude), crs=4326)
stops = gpd.GeoDataFrame(df[['id']], geometry=gpd.points_from_xy(df.dropoff_longitude, df.dropoff_latitude), crs=4326)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
i=133
a,b,current_route = starts.iloc[i].geometry, stops.iloc[i].geometry, [starts.iloc[i].geometry]

inter_roads = roads[roads.buffer(2e-4).intersects(a.buffer(2e-4))][['trafdir', 'geometry']]
endps = []

# this is used to find the endpoints of the roads based on road direction
for _,road in inter_roads.iterrows():
    if road.trafdir == 'FT':
        if not np.any([road.geometry.boundary[1].almost_equals(val) for val in endps]):
            endps.append(road.geometry.boundary[1]) 
    elif road.trafdir == 'TF':
        if not np.any([road.geometry.boundary[0].almost_equals(val) for val in endps]):
            endps.append(road.geometry.boundary[0])
    elif road.trafdir == 'TW':
        if not np.any([road.geometry.boundary[0].almost_equals(val) for val in endps]):
            endps.append(road.geometry.boundary[0])
        if not np.any([road.geometry.boundary[1].almost_equals(val) for val in endps]):
            endps.append(road.geometry.boundary[1])
    else:
        pass # this is a non vehicle road

res = []
for point in endps:

    if LineString([a,point]).length==0:
        continue
    if len(current_route)>1 and LineString(current_route).buffer(1e-4).intersects(point.buffer(1e-4)):
        continue

    # return all the candidates that do not intersect with the current route
    res.append((Point(point), Point(point).distance(b)))

In [63]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
from routing import route
from misc import *

In [30]:
vizpoints([val[0] for val in res]+[a])

In [81]:
from routing import route, _candidates
from misc import *

# i=7
for i in range(100):
    rt = route(starts.iloc[i].geometry, stops.iloc[i].geometry, roads, max_queue_len=999, result_len=1, routedif=1)
    # print(rt)
    # for k in range(5):
    #     for j in range(5):
    #         print(fastdtw([(a.x, a.y) for a in rt[k]],[(a.x, a.y) for a in rt[j]])[0])
    # viz([rt], starts.iloc[i:i+1], stops.iloc[i:i+1])
# input()
# clear_output()

72 0 0.0013593892669551237 (0, 0) inf


In [78]:
rt

In [75]:
viz_start_stop()(starts.iloc[i:i+1], stops.iloc[i:i+1])

In [47]:
rt

[[<shapely.geometry.point.Point at 0x29f21ee20>,